In [1]:
#import torch
import pandas as pd 
import numpy as np 
import cv2
import matplotlib.pyplot as plt 
import time
import os
from tqdm import tqdm

In [2]:
def read_image(imagepath):
    img = cv2.imread(imagepath,cv2.IMREAD_GRAYSCALE) 
    #plt.imshow(img)
    return img


In [3]:
def crop_text (img, laterality):
    if laterality == "L":
        text_area = img [:700, (img.shape[1]-700):]
        text_position = np.where(text_area !=0)
        x_min = min(text_position[1])
        return img[:, :-(700-x_min+10)]

    elif laterality == "R":
        text_area = img[:700,:700]
        text_position = np.where(text_area !=0)
        x_max = max(text_position[1])
        return img[:,(x_max+20):]

    return img



In [4]:
df = pd.read_csv("/home/single1/BACKUP/SamHUyen/mammo/sam/singleview_sam/updatedcsv_singleview.csv")
df["path"] = "/home/single1/BACKUP/SamHUyen/multi_view_mammo_classification/images/" + df["study_id"] + "/" + df["image_id"] + ".png" 

In [5]:
df["image_id"].nunique()

19276

In [6]:
def crop(img):
    """
    get region of interest
​
    args:
        img (numpy.ndarray): image of shape (H,W)
​
    return:
        img (numpy.ndarray): cropped image of shape (H', W')
    """
    top, bottom, left, right = get_boundaries(img)
    return img[top:bottom, left:right]


def get_boundaries(img):
    """
    return bbox in form [y_min, y_max, x_min, x_max]
    """
    mask = img != 0
    bounds = []
    for ax in [1,0]:
        region = mask.any(axis=ax)
        bounds.append((region.argmax(), mask.shape[1-ax] - region[::-1].argmax()))
    return [bounds[0][0], bounds[0][1], bounds[1][0], bounds[1][1]]

In [35]:
start = time.time()
path_save_img = "/home/single1/BACKUP/SamHUyen/multi_view_mammo_classification/crop-images-heuristic/" + df["image_id"] + ".png"
listimg = os.listdir("/home/single1/BACKUP/SamHUyen/multi_view_mammo_classification/crop-images-heuristic")
miss_img_path = []
miss_img_id = []
miss_img_gl = []
for i in tqdm(range(0,len(path_save_img))): 
    if (df["image_id"][i] + ".png") not in listimg:
        miss_img_path.append(df["path"][i])
        miss_img_id.append(df["image_id"][i])
        miss_img_gl.append(df["glaterality"][i])
        """
        img = read_image(df["path"][i])
        cropi = crop_text(img, df["glaterality"][i])
        img_crop = crop(cropi)
        cv2.imwrite(path_save_img[i],img_crop)
        """

end = time.time()
print("TIme ", end - start)


100%|██████████| 19276/19276 [00:02<00:00, 7749.21it/s]TIme  2.4987599849700928



In [92]:
start = time.time()
for i in tqdm(range(2,len(miss_img_path))): #len(miss_img_path)
    img = cv2.imread(miss_img_path[i],cv2.IMREAD_GRAYSCALE)
    cropi = crop_text(img, miss_img_gl[i])
    img_crop = crop(cropi)
    path_save_img = "/home/single1/BACKUP/SamHUyen/multi_view_mammo_classification/crop-images-heuristic/" + miss_img_id[i] + ".png"
    cv2.imwrite(path_save_img,img_crop)
end = time.time()
print("TIme ", end - start)

395/4052 [05:31<01:37,  6.77it/s]

 84%|████████▍ | 3396/4052 [05:32<02:03,  5.32it/s]

 84%|████████▍ | 3397/4052 [05:32<02:07,  5.15it/s]

 84%|████████▍ | 3398/4052 [05:32<01:56,  5.61it/s]

 84%|████████▍ | 3399/4052 [05:32<02:00,  5.42it/s]

 84%|████████▍ | 3400/4052 [05:32<02:20,  4.64it/s]

 84%|████████▍ | 3401/4052 [05:33<02:08,  5.08it/s]

 84%|████████▍ | 3403/4052 [05:33<01:44,  6.20it/s]

 84%|████████▍ | 3405/4052 [05:33<01:26,  7.52it/s]

 84%|████████▍ | 3407/4052 [05:33<01:13,  8.80it/s]

 84%|████████▍ | 3409/4052 [05:33<01:06,  9.71it/s]

 84%|████████▍ | 3411/4052 [05:33<01:00, 10.67it/s]

 84%|████████▍ | 3413/4052 [05:33<00:53, 11.90it/s]

 84%|████████▍ | 3415/4052 [05:33<00:47, 13.32it/s]

 84%|████████▍ | 3417/4052 [05:34<00:46, 13.69it/s]

 84%|████████▍ | 3419/4052 [05:34<00:49, 12.91it/s]

 84%|████████▍ | 3421/4052 [05:34<01:04,  9.73it/s]

 84%|████████▍ | 3423/4052 [05:34<00:58, 10.77it/s]

 85%|████████▍ | 3425/4052 [05:34<00:52, 11.83it/s]

 85%|██████

In [87]:
#list all images in folder images
list_studies = os.listdir("/home/single1/BACKUP/SamHUyen/multi_view_mammo_classification/images")
list_img_path = []
for std in list_studies:
    std_path = os.path.join("/home/single1/BACKUP/SamHUyen/multi_view_mammo_classification/images", std)
    img_list = os.listdir(std_path)
    for imgs in img_list:
        list_img_path.append(std_path + "/" +imgs)











In [88]:
len (list_img_path)

20270

In [90]:
#path not exist in list
remove_img_path = []
for path in miss_img_path:
    if path not in list_img_path:
        remove_img_path.append(path)
print(len(remove_img_path))

2


In [91]:
remove_img_path

['/home/single1/BACKUP/SamHUyen/multi_view_mammo_classification/images/1.2.410.200010.1073201.9145.1080045.1094364.1094364/1.3.12.2.1107.5.12.7.5054.30000020032400420414000000016.png',
 '/home/single1/BACKUP/SamHUyen/multi_view_mammo_classification/images/1.2.410.200010.1073404.7943.543090.551700.551700/1.3.12.2.1107.5.12.7.5054.30000019060700314957800000092.png']

In [46]:

test = os.listdir("/home/single1/BACKUP/SamHUyen/multi_view_mammo_classification/crop-images/crop_images")
unique_im = df["image_id"].unique() + ".png" 
miss_im = []
for i in tqdm(range(len(unique_im))):
    if unique_im[i] not in test:
        miss_im.append(i)

100%|██████████| 19276/19276 [00:02<00:00, 8071.30it/s]


In [47]:
miss_im

[1146, 1307, 2442, 5710, 7562, 9377, 9382, 15660, 16328, 16348, 18523, 18840]

In [97]:
len(os.listdir("/home/single1/BACKUP/SamHUyen/multi_view_mammo_classification/updatedata/images"))

8652

In [101]:
list_studies = os.listdir("/home/single1/BACKUP/SamHUyen/multi_view_mammo_classification/updatedata/images")
imgs = []
df = pd.read_csv("/home/single1/BACKUP/SamHUyen/mammo/sam/singleview_sam/updatedcsv_singleview.csv")
#txts = []
for study in list_studies:
    study_path = os.path.join("/home/single1/BACKUP/SamHUyen/multi_view_mammo_classification/updatedata/images", study)
    if study not in df["study_id"].values:
        imgs.append(study_path + "/") #study folder 
    #img_list = os.listdir(study_path)
    #for img in img_list:
    #    imgs.append(os.path.join(study_path,img))
    #    txts.append("/media/tungthanhlee/DATA/multi_view_mammo_classification/labels/" + str(img).replace("png","txt"))
print(len(imgs))

3835


In [102]:
df["study_id"].nunique()

4819